In [ ]:
# Example: My preferred approach
import os
# Settings -> Developer Settings -> Personal Access Tokens -> Token (classic)
os.environ['GITHUB_TOKEN'] = ""

GITHUB_USERNAME = "Codfishz"
REPO_NAME       = "ASR"
TOKEN = os.environ.get("GITHUB_TOKEN")
repo_url        = f"https://{TOKEN}@github.com/{GITHUB_USERNAME}/{REPO_NAME}.git"
!git clone {repo_url}

Cloning into 'ASR'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 72 (delta 31), reused 56 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (72/72), 11.79 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!cd {REPO_NAME} && git pull

Already up to date.


In [3]:
import os
os.chdir('ASR/Script')

In [4]:
%run "Datapreparation_YOLO.ipynb"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2
Dataset URL: https://www.kaggle.com/datasets/nirmalsankalana/crop-pest-and-disease-detection
License(s): CC0-1.0
✅ Total images found in dataset: 25220
Images before filter: 25220


Saving train: 100%|██████████| 20176/20176 [00:32<00:00, 627.48it/s]



train split summary:
  Total images: 20103



Saving val: 100%|██████████| 2522/2522 [00:03<00:00, 640.34it/s]



val split summary:
  Total images: 2511



Saving test: 100%|██████████| 2522/2522 [00:03<00:00, 642.08it/s]


test split summary:
  Total images: 2512

  Images after filter : 25126



In [5]:
!pip install ultralytics
!pip install torchinfo

import ultralytics
ultralytics.checks()

Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 43.7/235.7 GB disk)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
from ultralytics import YOLO
print("GPU available:", torch.cuda.is_available())

GPU available: True


# Active YOLO functions

In [7]:
def setup_active_dataset():

    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    active_train_dir = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR)
    os.makedirs(active_train_dir, exist_ok=True)

    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            os.makedirs(os.path.join(active_train_dir, cls), exist_ok=True)
    print(f"Complete creating train_dir {active_train_dir}")

    orig_val_dir = os.path.join(ORIGINAL_DATASET_DIR, VAL_SUBDIR)
    active_val_dir = os.path.join(ACTIVE_DATASET_DIR, VAL_SUBDIR)
    if os.path.exists(orig_val_dir):
        if os.path.exists(active_val_dir):
            shutil.rmtree(active_val_dir)
        shutil.copytree(orig_val_dir, active_val_dir)

    orig_test_dir = os.path.join(ORIGINAL_DATASET_DIR, TEST_SUBDIR)
    active_test_dir = os.path.join(ACTIVE_DATASET_DIR, TEST_SUBDIR)
    if os.path.exists(orig_test_dir):
        if os.path.exists(active_test_dir):
            shutil.rmtree(active_test_dir)
        shutil.copytree(orig_test_dir, active_test_dir)

In [8]:
def get_all_samples():

    samples = []
    orig_train_dir = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR)
    for cls in os.listdir(orig_train_dir):
        cls_path = os.path.join(orig_train_dir, cls)
        if os.path.isdir(cls_path):
            image_files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f))]
            for f in image_files:
                samples.append((cls, f))
    return samples

In [9]:
import shutil
def copy_samples(sample_list):

    for cls, file_name in sample_list:
        src_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        dst_path = os.path.join(ACTIVE_DATASET_DIR, TRAIN_SUBDIR, cls, file_name)
        if not os.path.exists(dst_path):
            shutil.copy(src_path, dst_path)

In [10]:
import random
def stratified_sample(all_samples, n_initial):

    samples_by_class = {}
    for cls, filename in all_samples:
        samples_by_class.setdefault(cls, []).append((cls, filename))

    stratified = []
    for cls, samples in samples_by_class.items():
        stratified.append(random.choice(samples))
    remaining_count = n_initial - len(stratified)
    if remaining_count > 0:
        remaining_samples = list(set(all_samples) - set(stratified))
        additional_samples = random.sample(remaining_samples, remaining_count)
        stratified.extend(additional_samples)

    return stratified



# Hyperparameters

In [11]:
ORIGINAL_DATASET_DIR = '/content/data_yolo'
ACTIVE_DATASET_DIR = '/content/data_active'

TRAIN_SUBDIR = 'train'
VAL_SUBDIR = 'val'
TEST_SUBDIR = 'test'
Num_Train = 20103
N_INITIAL = int(0.3 * Num_Train)
N_PER_PHASE = int(0.1 * Num_Train)
NUM_PHASES = 5

# TTA

In [23]:
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T

def compute_entropy(probs: np.ndarray) -> float:
    """Compute the entropy of a probability distribution."""
    return -np.sum(probs * np.log(probs + 1e-12))

from tqdm import tqdm
import os
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as T

def compute_entropy(probs: np.ndarray) -> float:
    """Compute the entropy of a probability distribution."""
    return -np.sum(probs * np.log(probs + 1e-12))

def select_tta_samples(model,
                       remaining_samples,
                       n_to_add,
                       K=5,
                       device='cuda'):
    """
    Select the top-n most uncertain samples using Test-Time Augmentation (TTA),
    showing a progress bar over remaining_samples.
    """
    # Base preprocessing (resize, tensor conversion)
    transform_base = T.Compose([
        T.Resize((640, 640)),
        T.ToTensor(),
    ])
    # A list of K augmentations (including the identity)
    aug_transforms = [
        transform_base,
        T.Compose([T.RandomHorizontalFlip(p=1.0), transform_base]),
        T.Compose([T.RandomVerticalFlip(p=1.0), transform_base]),
        T.Compose([T.RandomRotation(15), transform_base]),
        T.Compose([T.ColorJitter(brightness=0.2, contrast=0.2), transform_base]),
    ]

    model.eval()
    sample_uncertainties = []

    with torch.no_grad():
        # tqdm 进度条包装
        for cls, fname in tqdm(remaining_samples, desc="TTA AL sampling", unit="img"):
            img_path = os.path.join(ORIGINAL_DATASET_DIR, TRAIN_SUBDIR, cls, fname)
            img = Image.open(img_path).convert('RGB')
            probs_list = []

            for k in range(K):
                transform = aug_transforms[k % len(aug_transforms)]
                inp = transform(img).unsqueeze(0).to(device)

                result = model.predict(
                    inp,
                    verbose=False,
                    show=False
                )[0]

                probs_tensor = result.probs.data
                if isinstance(probs_tensor, torch.Tensor):
                    probs_np = probs_tensor.cpu().numpy()
                else:
                    probs_np = np.array(probs_tensor)
                probs_list.append(probs_np)

            avg_probs = np.mean(probs_list, axis=0)
            ent = compute_entropy(avg_probs)
            sample_uncertainties.append(((cls, fname), ent))

    # 取熵最高的 n_to_add 个样本
    sample_uncertainties.sort(key=lambda x: x[1], reverse=True)
    selected_samples = [s for (s, _) in sample_uncertainties[:n_to_add]]
    return selected_samples


# Example usage:
# new_samples = select_tta_samples(model, remaining_samples, n_to_add, K=5, device=device)


In [24]:
import csv

# Initialize dataset
setup_active_dataset()
all_samples = get_all_samples()

# Prepare CSV logging
log_path = "accuracy_log.csv"
with open(log_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Phase", "Num_Images", "Top-1 Accuracy", "Top-5 Accuracy"])

# Initial sampling and train
current_sample_list = stratified_sample(all_samples, N_INITIAL)
copy_samples(current_sample_list)

print("Initial Epoch")
model = YOLO('yolo11n-cls.pt')
results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name="active-phase0", project="runs/classify")

# Log initial accuracy
acc1 = results.top1
acc5 = results.top5

with open(log_path, "a", newline="") as f:
    writer = csv.writer(f)
    writer.writerow([0, len(current_sample_list), acc1, acc5])

# Compute remaining samples
remaining_samples = list(set(all_samples) - set(current_sample_list))

# Active Learning (Uncertainty Sampling)
for phase in range(1, NUM_PHASES):
    n_to_add = min(N_PER_PHASE, len(remaining_samples))
    if n_to_add <= 0:
        print("No more samples")
        break

    print(f"Phase {phase}: selecting top {n_to_add} uncertain samples...")

    # Load model from previous phase
    model = YOLO(f"runs/classify/active-phase{phase-1}/weights/last.pt")

    # Apply uncertainty sampling using TTA
    # Send to GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.model = model.model.to(device)

    new_samples = select_tta_samples(model, remaining_samples, n_to_add, K=5, device=device)
    copy_samples(new_samples)

    current_sample_list.extend(new_samples)
    remaining_samples = list(set(remaining_samples) - set(new_samples))

    print(f"Phase {phase}: add {n_to_add} samples, total sample number: {len(current_sample_list)}")

    # Train model for this phase
    results = model.train(data=ACTIVE_DATASET_DIR, epochs=1, imgsz=640, name=f"active-phase{phase}", project="runs/classify")

    # Log accuracy
    acc1 = results.top1
    acc5 = results.top5

    with open(log_path, "a", newline="") as f:
        writer = csv.writer(f)
        writer.writerow([phase, len(current_sample_list), acc1, acc5])

    print(f"Phase {phase+1} training completed")


Complete creating train_dir /content/data_active/train
Initial Epoch
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=classify, mode=train, model=yolo11n-cls.pt, data=/content/data_active, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/classify, name=active-phase06, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf

train: Scanning /content/data_active/train... 10297 images, 0 corrupt: 100%|██████████| 10297/10297 [00:00<?, ?it/s]
val: Scanning /content/data_active/val... 2511 images, 0 corrupt: 100%|██████████| 2511/2511 [00:00<00:00, 2998.22it/s]

val: WARNING ⚠️ /content/data_active/val/Tomato leaf curl/leaf curl185_.jpg: corrupt JPEG restored and saved
val: New cache created: /content/data_active/val.cache


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000385, momentum=0.9) with parameter groups 39 weight(decay=0.0), 40 weight(decay=0.0005), 40 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/classify/active-phase06
Starting training for 1 epochs...

      Epoch    GPU_mem       loss  Instances       Size


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:08<00:00,  9.37it/s]


                   all      0.763      0.984

1 epochs completed in 0.021 hours.
Optimizer stripped from runs/classify/active-phase06/weights/last.pt, 3.2MB
Optimizer stripped from runs/classify/active-phase06/weights/best.pt, 3.2MB

Validating runs/classify/active-phase06/weights/best.pt...
Ultralytics 8.3.109 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11n-cls summary (fused): 47 layers, 1,554,206 parameters, 0 gradients, 3.2 GFLOPs
train: /content/data_active/train... found 10297 images in 22 classes ✅ 
val: /content/data_active/val... found 2511 images in 22 classes ✅ 
test: /content/data_active/test... found 2512 images in 22 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 79/79 [00:08<00:00,  9.64it/s]


                   all      0.762      0.984
Speed: 0.5ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/active-phase06
Phase 1: selecting top 2010 uncertain samples...


TTA AL sampling:   2%|▏         | 347/14073 [00:28<18:55, 12.08img/s]


KeyboardInterrupt: 